## Set the enviroments.

#### Connect to slime.

In [1]:
(start-swank)

T

In [2]:
1

1

#### Add paths.

In [54]:
(leap.core::clear-path)
(add-path *default-pathname-defaults*)
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/")

#### Load the amber force field.

In [55]:
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")

(#<FORCE-FIELD ATOMTYPE_AMBER.DEF> #<FORCE-FIELD water_amber.def>
 #<FORCE-FIELD frcmod.ff14SB> #<FORCE-FIELD parm10.dat>
 #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [56]:
(source "leaprc.protein.ff14SB")

SIMPLE-WARNING: 
add-atom-types doesn't do anything
SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


T

In [57]:
(load-atom-type-rules "water_amber.def")

(#<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ff14SB>
 #<FORCE-FIELD parm10.dat> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>
 #<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ff14SB>
 #<FORCE-FIELD parm10.dat> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [58]:
(:= *ff* (leap.core::merged-force-field))

*FF*

## Create topology and coordinate file for Alanine

#### Load the alanine from Chemdraw file.

In [65]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [66]:
(build-hydrogens *chemagg*)

NIL

In [67]:
(cando::show-aggregate *chemagg*)

A Jupyter Widget

In [33]:
(nglv::show-aggregate *chemagg*)

SIMPLE-PROGRAM-ERROR: 
The symbol NGLV::SHOW-AGGREGATE has no function bound to it


NIL

#### Gather atoms to the center.

In [68]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

C      S


NIL

#### Set the bounding box.

In [64]:
(chem:set-property *chemagg* :bounding-box '(50.0 50.0 50.0))

NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [69]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

####  Assign force-field parameters and minimize the energy of the molecule.

In [70]:
(cando:jostle *chemagg*)

NIL

In [71]:
(defparameter *ee* (energy:minimize *chemagg*))

minimize-no-fail
======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-log(Alpha)--Dir-------------Energy-----------RMSforce
 minSDnP       0     1      -inf   0.0              0.000          16339.904
 minSDnP       0     2     -8.14   0.0        2528937.359          17449.828
 minSDnP       0     3    -10.03   0.0        2374907.581           5745.314
 minSDnP       0     4     -7.27   0.0        1751032.044           5515.470
DONE absolute force test:
forceRmsMag(3079.154505).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     5      -inf   0.0              0.000           3079.155
 minCGnP       0     6     -7.59  41.2        1270441.677           2693.444
 minCGnP       0     7     -6.67  56.4        1000959.503           3083.929
 minCGnP       0     8     -8.18  59.4         926844.380           3109.998
 minCGnP       0     9     -8.08  68.5         855390.461           3981.610
 minCGnP       0    10     -8.56  57.8   

*EE*

#### Visualize the minimized structure and demonstrate that the force field parameters lead to the construction of a chemically

#### reasonable structure.

In [14]:
(nglv::show-aggregate *chemagg*)

SIMPLE-PROGRAM-ERROR: 
The symbol NGLV::SHOW-AGGREGATE has no function bound to it


NIL

In [72]:
(cando::show-aggregate *chemagg*)

A Jupyter Widget

In [73]:
(save-mol2 *chemagg* "alanine.mol2")

Saving matter to /Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/alanine.mol2


NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [74]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

#### Save the topology file and coordinate file.

In [75]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg.top" "chemagg.crd" :assign-types nil)

atom1: #<ATOM :CH3/:C> atom2: #<ATOM :NH/:N> i1: 3 i2: 0
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 6 i2: 3
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 9 i2: 3
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 12 i2: 3
 atom1: #<ATOM :HH/:H> atom2: #<ATOM :NH/:N> i1: 15 i2: 0
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C/:C> i1: 18 i2: 21
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH31/:H> i1: 18 i2: 27
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH32/:H> i1: 18 i2: 30
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH33/:H> i1: 18 i2: 33
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 24 i2: 21
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 36 i2: 21
 atom1: #<ATOM :C/:C> atom2: #<ATOM :NH/:N> i1: 39 i2: 36
 atom1: #<ATOM :C/:C> atom2: #<ATOM :H/:H> i1: 39 i2: 42
 atom1: #<ATOM :C/:C> atom2: #<ATOM :CH3/:C> i1: 39 i2: 45
 atom1: #<ATOM :C/:C> atom2: #<ATOM :C_0/:C> i1: 39 i2: 48
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH31/:H> i1: 45 i2: 54
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH32/:H> i

0

## Create topology and coordinate file for Alanine and water atoms.

#### Load Tip3pfbox water box.

In [76]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

#### Make agggregate of the water box.

In [77]:
(print ~tip3pfbox)


#<AGGREGATE NIL> 

#<AGGREGATE NIL>

In [78]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [79]:
(nglv::show-aggregate *solvent*)

SIMPLE-PROGRAM-ERROR: 
The symbol NGLV::SHOW-AGGREGATE has no function bound to it


NIL

#### Assign the atom types of the force field to the water box.

In [80]:
(assign-atom-types *solvent*)

NIL

#### Add Tip3p water molecules to the alanine.

In [81]:
(leap.solvate::tool-solvate-and-shell *chemagg* *solvent* '(10.0 10.0 10.0))

Total bounding box for atom centers:  28.131195013744815d0 24.83926655882658d0 23.410263235708413d0


NIL

In [48]:
(nglv::show-aggregate *newagg*)

SIMPLE-PROGRAM-ERROR: 
The symbol NGLV::SHOW-AGGREGATE has no function bound to it


NIL

In [82]:
(show *chemagg*)

A Jupyter Widget

#### Save the topology and coordinate file. New aggregate contains water molecules and alanine.

In [83]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [84]:
(defparameter *e* (energy:minimize *chemagg*))

minimize-no-fail
======= Starting Steepest Descent Minimizer
DONE absolute force test:
forceRmsMag(0.979127).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
---Stage-Seconds--Step-log(Alpha)--Dir-------------Energy-----------RMSforce
 minCGnP       0     1      -inf   0.0              0.000              0.979
 minCGnP       0     2     -6.54  36.6              9.389              0.726
DONE absolute force test:
forceRmsMag(0.378793).LT.forceTolerance(0.500000)
minimize-no-fail
======= Starting Steepest Descent Minimizer
DONE absolute force test:
forceRmsMag(4.317184).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     3      -inf   0.0              0.000              4.317
 minCGnP       1     4     -7.07  41.2            137.950              3.775
 minCGnP       3     5     -6.24  66.2             81.136              6.074
 minCGnP       5     6     -6.70  68.6            -12.974              6.306
 minCGnP     

*E*

In [85]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg_water.top" "chemagg_water.crd" :assign-types nil)

atom1: #<ATOM :CH3/:C> atom2: #<ATOM :NH/:N> i1: 3 i2: 0
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 6 i2: 3
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 9 i2: 3
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 12 i2: 3
 atom1: #<ATOM :HH/:H> atom2: #<ATOM :NH/:N> i1: 15 i2: 0
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C/:C> i1: 18 i2: 21
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH31/:H> i1: 18 i2: 27
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH32/:H> i1: 18 i2: 30
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH33/:H> i1: 18 i2: 33
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 24 i2: 21
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 36 i2: 21
 atom1: #<ATOM :C/:C> atom2: #<ATOM :NH/:N> i1: 39 i2: 36
 atom1: #<ATOM :C/:C> atom2: #<ATOM :H/:H> i1: 39 i2: 42
 atom1: #<ATOM :C/:C> atom2: #<ATOM :CH3/:C> i1: 39 i2: 45
 atom1: #<ATOM :C/:C> atom2: #<ATOM :C_0/:C> i1: 39 i2: 48
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH31/:H> i1: 45 i2: 54
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH32/:H> i

0

#### Calculate each energy parameters.

In [31]:
(chem:get-energy (chem:get-stretch-component *e*))

1720.1189821006878d0

In [32]:
(chem:get-energy (chem:get-angle-component *e*))

0.30171081698364277d0

In [33]:
(chem:get-energy (chem:get-dihedral-component *e*))

5.456751705606037d0

In [34]:
(chem:get-energy (chem:get-nonbond-component *e*))

658.8024508583202d0

In [34]:
(apropos "superpose")

CHEM:ADD-SUPERPOSE-ATOM  Function
CHEM:CLEAR-SUPERPOSE-ATOMS  Function
CHEM:SET-FINAL-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-FINAL-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SUPERPOSE  Function
CANDO::SUPERPOSE-AGAINST-TEMPLATE  Function
CANDO::SUPERPOSE-ALL  Function
CHEM:SUPERPOSE-ALL-HEAVY-ATOMS  Function
CHEM:SUPERPOSE-ENGINE
CANDO::SUPERPOSE-ONE  Function
CHEM:SUPERPOSE-SELECTED-ATOMS
CHEM::SUPERPOSER


NIL

In [88]:
(:= w (show *chemagg*)) w

A Jupyter Widget

In [93]:
(nglv::picked w)

(("atom" ("index" . 20) ("residueIndex" . 2) ("resname" . "NIL_3")
  ("x" . 216.329) ("y" . 322.549) ("z" . -0.62) ("element" . "H")
  ("chainname" . "A") ("resno" . 3) ("serial" . 21) ("vdw" . 1.1)
  ("covalent" . 0.31) ("hetero" . 1) ("bfactor" . 0) ("altloc" . "")
  ("atomname" . "HH33")))

In [94]:
(defun distance (p1 p2) (geom:vlength (geom:v- p1 p2)))

DISTANCE

In [95]:
(:= *a* (let* ((picked (nglv::picked w))
       (atom (cdr (assoc "atom" picked :test #'string=)))
       (x (cdr (assoc "x" atom :test #'string=)))
       (y (cdr (assoc "y" atom :test #'string=)))
       (z (cdr (assoc "z" atom :test #'string=)))
       (pos (geom:vec x y z))
        resulta)
     (chem:map-atoms nil (lambda (a) 
                                 (when (< (distance pos (chem:get-position a)) 0.01) 
                                     (setf resulta a))) *chemagg*)
              resulta))

*A*

In [96]:
*a*

#<ATOM :HH33/:H>

In [97]:
(:= *p* (core:make-cxx-object 'chem:chem-info))

*P*

In [98]:
(chem:compile-smarts *p* "CC")

T

In [99]:
(chem:matches *p* *a*)

NIL